In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "ruoxinhuang" 
os.environ['KAGGLE_KEY'] = "f9d6d5aade71452503766f80812da73d"
!kaggle datasets download -d cmu11785/20fall-hw3p2
!unzip 20fall-hw3p2.zip

404 - Not Found
unzip:  cannot find or open 20fall-hw3p2.zip, 20fall-hw3p2.zip.zip or 20fall-hw3p2.zip.ZIP.


In [ ]:
import os
import numpy as np
from PIL import Image

from sklearn.metrics import pairwise

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
imageFolder_dataset = torchvision.datasets.ImageFolder(root='classification_data/train_data', 
                                                       transform=torchvision.transforms.ToTensor())

In [ ]:
imageFolder_dataloader = DataLoader(imageFolder_dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
imageFolder_dataset.__len__(), len(imageFolder_dataset.classes)

(380638, 4000)

In [ ]:
class Network(nn.Module):
    def __init__(self, num_feats, hidden_sizes, num_classes, feat_dim=4000):
        super(Network, self).__init__()
        
        self.hidden_sizes = [num_feats] + hidden_sizes + [num_classes]
        
        self.layers = []
        for idx, channel_size in enumerate(hidden_sizes):
            self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
                                         out_channels=self.hidden_sizes[idx+1], 
                                         kernel_size=3, stride=1, padding=1, bias=False))
            self.layers.append(nn.BatchNorm2d(self.hidden_sizes[idx+1]))
            self.layers.append(nn.ReLU(inplace=True))
            if(idx==3 or idx==5):
              self.layers.append(nn.Dropout(p=0.3))
            if(idx==1 or idx==2 or idx==4 or idx==6 or idx==7 or idx==8):
              self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            
        self.layers = nn.Sequential(*self.layers)
        
        # For creating the embedding to be passed into the Center Loss criterion
        self.linear_closs = nn.Linear(self.hidden_sizes[-2], feat_dim, bias=False)

    
    def forward(self, x, evalMode=False):
        output = x
        output = self.layers(output)
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        # Create the feature embedding for the Center Loss
        closs_output = self.linear_closs(output)
        #closs_output = self.relu_closs(closs_output)

        return closs_output

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [ ]:
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        print('epoch',epoch)
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

   
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
        else:
            test_verify(model, test_loader)
        scheduler.step()

def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total


def test_verify(model, test_loader):
    raise NotImplementedError

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='classification_data/train_data', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = DataLoader(train_dataset, batch_size=200, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='classification_data/val_data', 
                                               transform=torchvision.transforms.ToTensor())
dev_dataloader = DataLoader(dev_dataset, batch_size=200, 
                                             shuffle=True, num_workers=8)

In [ ]:
numEpochs = 5
num_feats = 3

learningRate = 15e-2
weightDecay = 5e-5

hidden_sizes = [64, 64, 64, 128, 128, 256, 256, 256, 256]
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
network = Network(num_feats, hidden_sizes, num_classes)
network.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)

In [ ]:
decayRate = 0.85
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [ ]:
network.train()
network.to(device)
train(network, train_dataloader, dev_dataloader)

epoch 0
Train Loss: 8.0033	Train Accuracy: 0.0010	Val Loss: 8.1386	Val Accuracy: 0.0003
epoch 1
Train Loss: 7.8296	Train Accuracy: 0.0015	Val Loss: 7.9520	Val Accuracy: 0.0014
epoch 2
Train Loss: 7.3568	Train Accuracy: 0.0032	Val Loss: 7.5112	Val Accuracy: 0.0015
epoch 3
Train Loss: 7.4162	Train Accuracy: 0.0032	Val Loss: 7.5616	Val Accuracy: 0.0025
epoch 4
Train Loss: 7.6524	Train Accuracy: 0.0033	Val Loss: 7.8380	Val Accuracy: 0.0021
epoch 5
Train Loss: 7.3809	Train Accuracy: 0.0078	Val Loss: 7.5767	Val Accuracy: 0.0046
epoch 6
Train Loss: 7.4217	Train Accuracy: 0.0066	Val Loss: 7.6073	Val Accuracy: 0.0053
epoch 7
Train Loss: 7.1421	Train Accuracy: 0.0095	Val Loss: 7.3469	Val Accuracy: 0.0071
epoch 8
Train Loss: 7.2777	Train Accuracy: 0.0098	Val Loss: 7.4943	Val Accuracy: 0.0076
epoch 9
Train Loss: 7.5456	Train Accuracy: 0.0082	Val Loss: 7.7623	Val Accuracy: 0.0050


In [ ]:
f = open("verification_pairs_test.txt", "r")
lines = f.read().split('\n')
lines = np.array(lines)

In [ ]:
lines = np.char.split(lines, sep =' ') 

In [ ]:
for line in lines:
  img = Image.open(line[0])
  img = torchvision.transforms.ToTensor()(img)
  img = img.reshape((1,3,64,64))
  network.eval()
  feats = img.to(device)
  output1 = network(feats)

  img = Image.open(line[1])
  img = torchvision.transforms.ToTensor()(img)
  img = img.reshape((1,3,64,64))
  network.eval()
  feats = img.to(device)
  output2 = network(feats)

  score = pairwise.cosine_similarity(output1.cpu().detach().numpy(),output2.cpu().detach().numpy())
  line.append(score)

AttributeError: ignored

In [ ]:
import csv

with open('submission3.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(('Id','Category'))
    for i in range(lines.shape[0]-1):
        csvwriter.writerow((lines[i][0]+' '+lines[i][1],lines[i][2][0][0]))